In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import pyrosetta
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score

In [3]:
# cd ~/Documents/0_git/PyRosettaCluster/tutorials
# rsync -auP aloshbaugh@35.168.122.119:/shared/home/pce-shared/environments/jupyterlab.yml .
# conda deactivate
# conda env create -f jupyterlab.yml

# To install pyrosetta cluster:
# cd PyRosettaCluster
# In the dir with setup.py (git repo PyRosettaCluster), do pip install -e . --user

from pyrosettacluster import PyRosettaCluster, get_instance_kwargs

Simplest possible example
XML protocol - keep it simple - FastRelax?
Use a SingleoutputRosettaScriptsTask -- https://github.com/proteininnovation/Rosetta-PyData_Integration/blob/master/ddg_analysis/ddG_pssm.ipynb
Single pose - Top7 (PDB accession code 1QYS)
Reproduce the output


In [5]:
input_protocol = """
<ROSETTASCRIPTS>
  <TASKOPERATIONS>
    <RestrictToRepacking name="only_pack"/>
  </TASKOPERATIONS>

  <MOVERS>
    <PackRotamersMover name="pack" task_operations="only_pack" />
  </MOVERS>
  
  <PROTOCOLS>
    <Add mover="pack"/>
  </PROTOCOLS>
</ROSETTASCRIPTS>
"""

In [6]:
input_relax = rosetta_scripts.SingleoutputRosettaScriptsTask(input_protocol)
# Syntax check via setup
input_relax.setup()

INFO:pyrosetta.distributed:maybe_init performing pyrosetta initialization: {'extra_options': '-out:levels all:warning', 'silent': True}
INFO:pyrosetta.rosetta:Found rosetta database at: /Users/aloshbaugh/miniconda3/envs/pyrosetta2/lib/python3.7/site-packages/pyrosetta/database; using it....
INFO:pyrosetta.rosetta:PyRosetta-4 2019 [Rosetta PyRosetta4.conda.mac.python37.Release 2020.01+release.17852696da60cbdc09f8d8371140f4880d1e41f3 2019-12-29T14:09:34] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [12]:
pdb_string = !curl https://files.rcsb.org/download/1qys.pdb
pdb_string = "\n".join(pdb_string)
ppose = score.ScorePoseTask()(io.pose_from_pdbstring(pdb_string))
wpose = pyrosetta.distributed.packed_pose.to_pose( ppose )

INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue LYS 13
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD  on residue LYS 13
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CE  on residue LYS 13
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  NZ  on residue LYS 13
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CG  on residue PHE 15
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD1 on residue PHE 15
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CD2 on residue PHE 15
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CE1 on residue PHE 15
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  CE2 on residue PHE 15
INFO:rosetta:core.conformation.Conformation: {0} [ WARNING ] missing heav

In [14]:
input_pose = input_relax(wpose)

In [ ]:
def mutate_residue(input_pose, res_index, new_aa, res_label = None):
    import pyrosetta.rosetta.core.pose as pose
    
    work_pose = packed_pose.to_pose(input_pose)
    
    # Annotate strucure with reslabel, for use in downstream protocol
    # Add parameters as score, for use in downstream analysis
    if res_label:
        work_pose.pdb_info().add_reslabel(res_index, res_label)
        pose.setPoseExtraScore(work_pose, "mutation_index", res_index)
        pose.setPoseExtraScore(work_pose, "mutation_aa", new_aa)
    
    if len(new_aa) == 1:
        new_aa = str.upper(Bio.SeqUtils.seq3(new_aa))
    assert new_aa in map(str.upper, IUPACData.protein_letters_3to1)
    
    protocol = """
<ROSETTASCRIPTS>
    <MOVERS>
        <MutateResidue name="mutate" new_res="%(new_aa)s" target="%(res_index)i" />
    </MOVERS>
    <PROTOCOLS>
        <Add mover_name="mutate"/>
    </PROTOCOLS>
</ROSETTASCRIPTS>
    """ % locals()
    
    return rosetta_scripts.SingleoutputRosettaScriptsTask(protocol)(work_pose)